In [6]:
import torch
import torch.nn as nn
from networks import AttnDecoderRNN, CNN
from model import CHARACTERModel

In [21]:
class CAM_normD(nn.Module):
    def __init__(self,nclass,channel_size,hidden_size,output_size,dropout_p=0.1,max_length = 64,D_ch =16, nWriter = 1300,iam = False):
        super(CAM_normD,self).__init__()
        self.encoder =CNN(channel_size)
        self.hidden_size = hidden_size
        
        self.decoder_forradical = AttnDecoderRNN(hidden_size, output_size, dropout_p,max_length)
        # self.decoderfeat_forradical =nn.Sequential(
        #     nn.Conv2d(256, 128, 3, 1, 1), nn.InstanceNorm2d(128), nn.PReLU(), nn.AvgPool2d(2, 2),
        #     nn.Conv2d(128, 64, 3, 1, 1), nn.InstanceNorm2d(128), nn.PReLU(), nn.MaxPool2d(2, 2),
        #     nn.Conv2d(64, 16, 3, 1, 1), nn.InstanceNorm2d(16), nn.PReLU(),
        #     nn.Conv2d(16, 1, 3, 1, 1)
        #     )        

        self.decoderfeat_forradical0 =nn.Sequential(
            nn.Conv2d(256, 128, 3, 1, 1), nn.InstanceNorm2d(128), nn.PReLU(), nn.AvgPool2d(2, 2),
            nn.Conv2d(128, 64, 3, 1, 1)
            )        

        self.decoderfeat_forradical1 =nn.Sequential(
            nn.InstanceNorm2d(128), nn.PReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 16, 3, 1, 1), nn.InstanceNorm2d(16), nn.PReLU(),
            nn.Conv2d(16, 1, 3, 1, 1)
            )        

        

    def initHidden(batch_size,hidden_size):
        result = torch.autograd.Variable(torch.zeros(1, batch_size, hidden_size))
        return result

    def forward(self, image, text_radical, length_radical):
        
        encode = self.encoder(image)
        b, c, _, _ = encode.size() #batch,256
        
        # out, bottleneck_out = self.unetD(image)
        loss_forradical,new_encode= self.decoder_forradical(encode,image,text_radical,length_radical)
        # pred_radical = self.decoderfeat_forradical(new_encode)
        pred_radical0 = self.decoderfeat_forradical0(new_encode)
        pred_radical1 = self.decoderfeat_forradical1(pred_radical0)
        pred_radical = (pred_radical0, pred_radical1)
        
        return pred_radical, loss_forradical

In [8]:
from options import TrainOptions
opt = TrainOptions()
from x_dataset import CGGANDataset

dataset = CGGANDataset(args=opt)
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=opt.batch_size,
    shuffle=True, sampler=None, drop_last=True, num_workers=int(opt.num_threads))

In [9]:
nclass = opt.n_alphabet+1
channel_size = opt.input_nc
hidden_size = opt.hidden_size
output_size = opt.n_alphabet+2
dropout_p = opt.dropout_p
max_length = opt.max_length
D_ch = opt.D_ch
nWriter = opt.num_writer
norm = opt.norm
init_type = opt.init_type 
init_gain = opt.init_gain
iam = False

In [22]:
camD = CAM_normD(nclass,channel_size,hidden_size,output_size,dropout_p,max_length,D_ch,nWriter,iam)
camD = camD.cuda()

In [11]:
model = CHARACTERModel(opt=opt)

Param count for Ds initialized parameters: 19541696
Param count for Ds initialized parameters: 20591296
Param count for Ds initialized parameters: 27289027


In [ ]:
# self.netD(self.img_print2write, self.new_lexicon_B, self.new_lexicon_B_length) 

In [13]:
data = next(iter(train_loader))

In [14]:
model.set_input(data)

In [15]:
model.forward()

In [17]:
model.img_print2write.shape

torch.Size([32, 3, 128, 128])

In [31]:
a = camD(model.img_print2write, model.new_lexicon_B, model.new_lexicon_B_length)

In [40]:
a[0][0].shape

torch.Size([160, 64, 4, 4])

In [75]:
b = nn.InstanceNorm2d(64,track_running_stats=True).cuda()
b.eval()

InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)

In [76]:
b(a[0][0])

tensor([[[[-0.0839,  0.0087, -0.0390, -0.2772],
          [-0.3285, -0.5137, -0.1781, -0.2795],
          [-0.3589, -0.1092,  0.0340, -0.2758],
          [-0.1578, -0.4120,  0.0130, -0.2952]],

         [[-0.2020, -0.1731, -0.2308, -0.2204],
          [-0.1027, -0.0761,  0.1375,  0.0179],
          [-0.0535, -0.2236, -0.6262, -0.3130],
          [-0.1018,  0.0040, -0.0196, -0.0463]],

         [[-0.0065, -0.2700, -0.0223, -0.1415],
          [ 0.0446, -0.7038,  0.0298, -0.1190],
          [-0.4223, -0.0909,  0.0758,  0.1424],
          [-0.1900, -0.2101,  0.2156, -0.1500]],

         ...,

         [[ 0.1089,  0.1299,  0.0216,  0.2868],
          [-0.2605, -0.0535,  0.4585,  0.5000],
          [-0.1042,  0.1655,  0.3769,  0.2644],
          [ 0.0191,  0.3691,  0.3372, -0.0112]],

         [[ 0.0536,  0.0084, -0.0070,  0.1037],
          [ 0.1634,  0.2899,  0.2397,  0.1969],
          [ 0.0274,  0.4197,  0.4157,  0.0857],
          [ 0.1810,  0.3479,  0.3841, -0.0785]],

         [[-0.1

In [69]:
model.netD

CAM_normD(
  (encoder): CNN(
    (cnn): Sequential(
      (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): PReLU(num_parameters=1)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(128, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): PReLU(num_parameters=1)
      (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (12): Conv2d(160, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1,

In [62]:
bb = b(a[0][0])

In [63]:
bb.shape

torch.Size([160, 64, 4, 4])

In [55]:
import torch.nn.functional as F

In [59]:
F.instance_norm(a[0][0])

tensor([[[[ 0.7497,  1.3311,  1.0321, -0.4640],
          [-0.7863, -1.9488,  0.1586, -0.4786],
          [-0.9767,  0.5907,  1.4903, -0.4554],
          [ 0.2860, -1.3105,  1.3587, -0.5769]],

         [[-0.3722, -0.2003, -0.5431, -0.4814],
          [ 0.2179,  0.3757,  1.6443,  0.9341],
          [ 0.5098, -0.5005, -2.8917, -1.0317],
          [ 0.2232,  0.8516,  0.7114,  0.5530]],

         [[ 0.4913, -0.7170,  0.4188, -0.1276],
          [ 0.7255, -2.7057,  0.6576, -0.0245],
          [-1.4151,  0.1043,  0.8684,  1.1736],
          [-0.3502, -0.4423,  1.5095, -0.1667]],

         ...,

         [[-0.2557, -0.1565, -0.6676,  0.5843],
          [-1.9994, -1.0221,  1.3949,  1.5905],
          [-1.2616,  0.0118,  1.0098,  0.4787],
          [-0.6795,  0.9729,  0.8224, -0.8227]],

         [[-0.7963, -1.0877, -1.1872, -0.4728],
          [-0.0873,  0.7286,  0.4044,  0.1284],
          [-0.9649,  1.5664,  1.5404, -0.5892],
          [ 0.0261,  1.1031,  1.3362, -1.6483]],

         [[ 0.0